In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf

In [2]:
#displaying stocks using yfinance
df = yf.download(tickers = "AAPL GOOG NFLX AMZN", period = "1y",interval = "1d",ignore_tz = True,prepost = False)['Adj Close']
df

[*********************100%***********************]  4 of 4 completed


,AAPL,AMZN,GOOG,NFLX
Date,,,,
2022-04-07,171.103500,157.784500,136.464996,362.149994
2022-04-08,169.065842,154.460495,134.010498,355.880005
2022-04-11,164.751968,151.121994,129.796494,348.000000
2022-04-12,166.650482,150.787506,128.374496,344.100006
2022-04-13,169.373993,155.541000,130.285995,350.429993
...,...,...,...,...
2023-03-31,164.899994,103.290001,104.000000,345.480011
2023-04-03,166.169998,102.410004,104.910004,348.279999
2023-04-04,165.630005,103.949997,105.120003,346.750000


In [3]:
#calculation of returns
returns = df.pct_change()
returns

,AAPL,AMZN,GOOG,NFLX
Date,,,,
2022-04-07,NaN,NaN,NaN,NaN
2022-04-08,-0.011909,-0.021067,-0.017986,-0.017313
2022-04-11,-0.025516,-0.021614,-0.031445,-0.022142
2022-04-12,0.011523,-0.002213,-0.010956,-0.011207
2022-04-13,0.016343,0.031524,0.014890,0.018396
...,...,...,...,...
2023-03-31,0.015644,0.012647,0.026451,0.020832
2023-04-03,0.007702,-0.008520,0.008750,0.008105
2023-04-04,-0.003250,0.015038,0.002002,-0.004393


In [4]:
#gives every possible covariance between 2 stocks so we get a matrix type of structure
cov_matrix_annual = returns.cov()*252
cov_matrix_annual

,AAPL,AMZN,GOOG,NFLX
AAPL,0.119952,0.116928,0.104909,0.111736
AMZN,0.116928,0.230693,0.143564,0.171458
GOOG,0.104909,0.143564,0.159216,0.124726
NFLX,0.111736,0.171458,0.124726,0.414324


In [5]:
#number of random weights we will be generating for simulation,more the value, more the value closer to theoritical value
num_port= 10000

#initializing all the required arrays
all_weights = np.zeros((num_port, 4))
returnss = np.zeros(num_port)
volatilities = np.zeros(num_port)
sharpe_ratios = np.zeros(num_port)

In [6]:
#in this for loop we are loading the above initialized arrays with their respective values using random weights
for ind in range(num_port):

    weights = np.array(np.random.random(4))
    weights = weights / np.sum(weights)

    all_weights[ind, :] = np.round_(weights, decimals = 3)

    #for small returns log returns are same as returns
    returnss[ind] = np.sum(returns.mean()*weights)*252
    volatilities[ind] = np.sqrt(np.dot(weights.T,np.dot(cov_matrix_annual,weights)))
    sharpe_ratios[ind] = returnss[ind]/volatilities[ind]

    
#converting the above data we calculated to a dataframe for better use
simulations_data = [returnss, volatilities, sharpe_ratios, all_weights]
simulations_df = pd.DataFrame(data=simulations_data).T

simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

simulations_df = simulations_df.infer_objects()

In [7]:
# Finding the location of maximum sharpe ratio from sharpe ratio column and returning its value along with its allocated weights.
max_sharpe = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Finding the location of minimum volatility from volatility column and returning its value along with its allocated weights.
min_vol = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('')
print('='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(max_sharpe)
print('-'*80)

print('')
print('='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(min_vol)
print('-'*80)


MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns                                  0.120415
Volatility                               0.527539
Sharpe Ratio                             0.228257
Portfolio Weights    [0.252, 0.007, 0.002, 0.739]
Name: 6558, dtype: object
--------------------------------------------------------------------------------

MIN VOLATILITY:
--------------------------------------------------------------------------------
Returns                                 -0.004422
Volatility                               0.342768
Sharpe Ratio                            -0.012901
Portfolio Weights    [0.831, 0.023, 0.122, 0.023]
Name: 6189, dtype: object
--------------------------------------------------------------------------------
